In [1]:
from utilities import * 
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
%load_ext autoreload
%autoreload 2
nest_asyncio.apply()

df1_path = '../dataset/result/nist_form/20230905081756/test_pred.csv'
df2_path = '../dataset/raw/nist/aws_cleaned.csv'

## Donut result:

In [2]:
df1 = pd.read_csv(df1_path)
df1.fillna('None', inplace=True)
print(df1.shape)
df1.head()

(500, 6)


,DATE,CITY,STATE,ZIP,image_name,text_sequence
0,08/02/89,FLINT,Mi,48504,f0000_14.png,None
1,8-7-89,Allendale,MI,49401,f0001_41.png,None
2,08-03-89,Holland,Mi,49424,f0004_09.png,None
3,8/3/89,ST. CLAIR SHORES,MI.,48082,f0006_12.png,None
4,8/2/89,EAST LANSING,MicHIGAN,48823,f0009_06.png,None


## TT result:


In [3]:
df2 = pd.read_csv(df2_path)
df2.fillna('None', inplace=True)
print(df2.shape)
df2.head()

(2100, 9)


,image_name,DATE,DATE__CONFIDENCE,CITY,CITY__CONFIDENCE,STATE,STATE_CONFIDENCE,ZIP,ZIP_CONFIDENCE
0,f0000_14.png,08/02/89,88.97464,FLiNT,86.739792,Mi,80.102089,48504,73.823257
1,f0001_41.png,8-7-89,87.28508,Allendale,89.946899,MI,82.12265,49401,77.600266
2,f0002_01.png,8-3-89,81.038841,MINDEN CITY,80.873611,mi,35.721474,48456,37.457157
3,f0003_42.png,08-03-89,87.377403,Charlotte,80.985222,MI,77.612007,48813,71.598038
4,f0004_09.png,08-03-89,89.594048,Holland,80.345238,Mi,62.762756,49424,58.702671


# Donut + TT + GPU

In [9]:
def clean_dicts_(input_image_name):
    
    # collect the JSONS
    df_1 = df1[df1.image_name == input_image_name]
    df_2 = df2[df2.image_name == input_image_name]
    
    # make sure they both JSONS have all the keys
    dict1 = init_json(df_1)
    dict2 = init_json(df_2)
    
    # creat the promet
    prompt = create_prompt(dict1, dict2, prefix_prompt, postfix_prompt, example_dict)
    
    dict_ = get_completion(prompt)
    dict3 = json.loads(dict_)
    
    get_completion(prompt)
    
    return dict1, dict2, dict3

async def run_function_with_retry(def_to_run, 
                                  input_image_name,
                                  max_retry_count= 2,
                                  retry_interval= 1):

    for retry in range(max_retry_count):
        loop = asyncio.get_event_loop()
        with concurrent.futures.ThreadPoolExecutor() as pool:
            try:
                future = pool.submit(def_to_run, input_image_name)
                result = await asyncio.wait_for(loop.run_in_executor(None, future.result), timeout=10)
                return result 
                break
            except asyncio.TimeoutError:
                print("Function didn't finish in time. Retrying...")
            except concurrent.futures.TimeoutError:
                print("Concurrent Future didn't finish in time. Retrying...")
            await asyncio.sleep(retry_interval)
    else:
        print("Function didn't finish within retry attempts.")

In [10]:
error_list = []
result_list = []
for i in tqdm(df1.image_name.to_list()):
    try:
        loop = asyncio.get_event_loop()
        dict1, dict2, dict3  = loop.run_until_complete(run_function_with_retry(clean_dicts_, i))
        dict3['image_name'] = i
        result_list.append(dict3)

    except Exception as e:
        print(str(e))
        error_list.append(i)
    

 25%|██████████████████████████████████████▌                                                                                                                      | 123/500 [07:08<21:38,  3.44s/it]

Function didn't finish in time. Retrying...


 26%|███████████████████████████████████████▉                                                                                                                   | 129/500 [17:29<3:28:32, 33.73s/it]

Function didn't finish in time. Retrying...


 35%|██████████████████████████████████████████████████████▎                                                                                                      | 173/500 [30:01<18:02,  3.31s/it]

Function didn't finish in time. Retrying...


 75%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                       | 375/500 [52:49<07:22,  3.54s/it]

Function didn't finish in time. Retrying...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [1:09:50<00:00,  8.38s/it]


In [11]:
df_gpt = pd.DataFrame(result_list)
print(df_gpt.shape)
df_gpt.head(2)

(500, 5)


,DATE,CITY,STATE,ZIP,image_name
0,08/02/89,FLINT,MI,48504,f0000_14.png
1,8-7-89,Allendale,MI,49401,f0001_41.png


In [ ]:
df_gpt.to_csv('../dataset/raw/nist/donut-tt-gpt.csv', index = False)

In [84]:
error_list

['f0050_10.png',
 'f0067_25.png',
 'f0067_25_11.png',
 'f0067_25_13.png',
 'f0067_25_15.png',
 'f0067_25_17.png',
 'f0067_25_18.png',
 'f0067_25_28.png',
 'f0067_25_3.png']

In [5]:
input_image_name = 'f0050_10.png'

In [6]:
df_1 = df1[df1.image_name == input_image_name]
df_2 = df2[df2.image_name == input_image_name]

# make sure they both JSONS have all the keys
dict1 = init_json(df_1)
dict2 = init_json(df_2)

# creat the promet
prompt = create_prompt(dict1, dict2, prefix_prompt, postfix_prompt, example_dict)

dict_ = get_completion(prompt)

In [7]:
dict_

'{"DATE": "Aug. 4,1989", "CITY": "Marietta", "STATE": "Ohio", "ZIP": "45754"}'

In [ ]:
prompt